In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 22:04:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
pyspark.__version__

'3.5.4'

In [2]:
df  = spark.read.parquet('data/raw/yellow/2024/10')

In [4]:
df.head(5)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 30, 44), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 48, 26), passenger_count=1, trip_distance=3.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=246, payment_type=1, fare_amount=18.4, extra=1.0, mta_tax=0.5, tip_amount=1.5, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=24.9, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 12, 20), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 25, 25), passenger_count=1, trip_distance=2.2, RatecodeID=1, store_and_fwd_flag='N', PULocationID=48, DOLocationID=236, payment_type=1, fare_amount=14.2, extra=3.5, mta_tax=0.5, tip_amount=3.8, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=23.0, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 4, 46), tpep_dropoff_datetime=datetime.datetime(2

In [5]:
df = df.repartition(4)

In [6]:
df.write.parquet('data/homework/yellow_2024_10/')

In [8]:
df.registerTempTable('yellow_trip_data')

/home/aadarsh/spark/spark-3.5.4-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [11]:
spark.sql("""
SELECT 
    COUNT(*) AS Number_of_trips
FROM 
    yellow_trip_data
WHERE
    TO_DATE(tpep_pickup_datetime, 'yyyy-MM-dd') = '2024-10-15'
""").show()

+---------------+
|Number_of_trips|
+---------------+
|         128893|
+---------------+



In [12]:
spark.sql("""
SELECT 
    MAX(DATEDIFF(second, tpep_pickup_datetime, tpep_dropoff_datetime)/3600)
FROM 
    yellow_trip_data
""").show()

+--------------------------------------------------------------------------------+
|max((timestampdiff(second, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600))|
+--------------------------------------------------------------------------------+
|                                                              162.61777777777777|
+--------------------------------------------------------------------------------+



In [16]:
df_zones = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')

In [17]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [18]:
df_zones.registerTempTable('taxi_zones')

/home/aadarsh/spark/spark-3.5.4-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [20]:
spark.sql("""
SELECT 
    taxi_zones.Zone, COUNT(*) AS Number_of_Pickups
FROM 
    yellow_trip_data
JOIN
    taxi_zones
ON 
    yellow_trip_data.PULocationID = taxi_zones.LocationID
GROUP BY
    taxi_zones.Zone
ORDER BY
    Number_of_Pickups ASC
""").show()

+--------------------+-----------------+
|                Zone|Number_of_Pickups|
+--------------------+-----------------+
|Governor's Island...|                1|
|       Rikers Island|                2|
|       Arden Heights|                2|
|         Jamaica Bay|                3|
| Green-Wood Cemetery|                3|
|   Rossville/Woodrow|                4|
|       West Brighton|                4|
|       Port Richmond|                4|
|Eltingville/Annad...|                4|
|Charleston/Totten...|                4|
|         Great Kills|                6|
|        Crotona Park|                6|
|     Mariners Harbor|                7|
|Heartland Village...|                7|
|Saint George/New ...|                9|
|             Oakwood|                9|
|New Dorp/Midland ...|               10|
|       Broad Channel|               10|
|         Westerleigh|               12|
|     Pelham Bay Park|               12|
+--------------------+-----------------+
only showing top